In [1]:
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from keras.datasets import cifar10
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

C:\Users\hiroa\Anaconda3\envs\py27\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


170500096/170498071 [==============================] - 838s 5us/step


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage
from sklearn.model_selection import train_test_split
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])

In [3]:
# floatに変換しておく
# 正規化するために
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255.0
X_test /= 255.0
# RGB各要素について0-255の値で色を表現する、例えば白は（R 255、G 255、B 255）で表現

# one-hot vector に変換したほうが都合が良い
# たとえば 4 ではなくて [0, 0, 0, 0, 1, 0, 0, 0, 0, 0] のようなデータにする
# こうしないとクラス分類ではなくて回帰として扱われてしまいうまくいかない
n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [4]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
 #   x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
 #   x = _resblock(n_filters=128)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
  

    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 3)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   256         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [5]:
#checkpoint
callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 50
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
  17/1000 [..............................] - ETA: 2:36:11 - loss: 2.2819 - acc: 0.1565

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epoch = np.arange(len(h.history['acc']))

# accuracy と validation accuracy の推移をプロットする
plt.title('Accuracy')
plt.ylim(0.0, 1.0)
plt.plot(epoch, h.history['acc'], label='train')
plt.plot(epoch, h.history['val_acc'], label='validation')
plt.legend()
plt.show()

In [ ]:
results = model.predict(X_test)

pos = 1
plt.figure(figsize=(16,24), dpi=60)

for i in range(100):
  img = toimage(X_test[i])
  label = results[i].argmax()
  plt.subplot(10, 10, pos)
  plt.imshow(img)
  plt.axis('off')
  plt.title(cifar10_labels[label])
  pos += 1        

plt.show()